In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### GETTING DATASET AND YAML (DATASET FIXED FOR WRONG LABELLINGS MANUALLY)

In [1]:
!gdown --id "10O-6uKWup9sCL3YoTOHox14w85WHnWzJ"

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=10O-6uKWup9sCL3YoTOHox14w85WHnWzJ
From (redirected): https://drive.google.com/uc?id=10O-6uKWup9sCL3YoTOHox14w85WHnWzJ&confirm=t&uuid=b3471d01-fbeb-4747-8781-108983005cff
To: /content/dataset.zip
100% 573M/573M [00:08<00:00, 70.9MB/s]


In [2]:
!gdown --id "1xX_bCZdtL4HLd8tFXo7cRf1FeZPxJAy_"

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1xX_bCZdtL4HLd8tFXo7cRf1FeZPxJAy_
To: /content/data.yaml
100% 265/265 [00:00<00:00, 1.30MB/s]


In [3]:
!cat data.yaml

train: /content/dataset/images/train
val: /content/dataset/images/val

nc: 14
names:
  [
    car,
    bike,
    auto,
    rickshaw,
    cycle,
    bus,
    minitruck,
    truck,
    van,
    taxi,
    motorvan,
    toto,
    train,
    boat,
  ]

In [4]:
!unzip dataset.zip
!pip install ultralytics optuna

Streaming output truncated to the last 5000 lines.
  inflating: dataset/images/train/rainy day (134).jpg  
  inflating: dataset/images/train/rainy day (135).jpg  
  inflating: dataset/images/train/rainy day (136).jpg  
  inflating: dataset/images/train/rainy day (137).jpg  
  inflating: dataset/images/train/rainy day (138).jpg  
  inflating: dataset/images/train/rainy day (139).jpg  
  inflating: dataset/images/train/rainy day (14).jpg  
  inflating: dataset/images/train/rainy day (140).jpg  
  inflating: dataset/images/train/rainy day (141).jpg  
  inflating: dataset/images/train/rainy day (142).jpg  
  inflating: dataset/images/train/rainy day (143).jpg  
  inflating: dataset/images/train/rainy day (144).jpg  
  inflating: dataset/images/train/rainy day (145).jpg  
  inflating: dataset/images/train/rainy day (146).jpg  
  inflating: dataset/images/train/rainy day (147).jpg  
  inflating: dataset/images/train/rainy day (148).jpg  
  inflating: dataset/images/train/rainy day (149).jpg 

### USING OPTUNA TO TUNE LEARNING RATE AND NUMBER OF EPOCHS (primary focus)

## USING SDG OPTIMIZER (best result 0.527 mAP50 and 0.267 mAP50-95), LATER IMPROVED BY USING ADAM OPTIMIZER

In [ ]:
import optuna
from ultralytics import YOLO
from google.colab import drive
import shutil
import os

def objective(trial):
    # Define the hyperparameters to optimize
    hyperparameters = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-6, 1e-2),
        "epochs": trial.suggest_int("epochs", 5, 10),
        'batch_size': trial.suggest_categorical('batch_size', [8, 16]),
        'img_size': trial.suggest_categorical('img_size', [416, 512, 640]),
        'warmup_epochs': trial.suggest_int('warmup_epochs', 0, 3),
        'fliplr': trial.suggest_uniform('fliplr', 0.0, 0.5),
        'mosaic': trial.suggest_uniform('mosaic', 0.0, 1.0),
    }

    # Create the model
    model = YOLO('yolov10m.pt')  # or 'yolov8n.pt' for a pretrained model

    # Train the model
    results = model.train(
        data='data.yaml',
        epochs=hyperparameters['epochs'],
        imgsz=hyperparameters['img_size'],
        batch=hyperparameters['batch_size'],
        lr0=hyperparameters['learning_rate'],
        warmup_epochs=hyperparameters['warmup_epochs'],
        fliplr=hyperparameters['fliplr'],
        mosaic=hyperparameters['mosaic'],
        optimizer='SGD',
    )

    # source_folder = '/content/runs'
    # destination_folder = '/content/drive/My Drive/ML/runs'

    # # Check if the destination folder already exists
    # if os.path.exists(destination_folder):
    #     # Remove the existing folder in Google Drive to avoid conflicts
    #     shutil.rmtree(destination_folder)

    # # Copy the updated 'runs' folder to Google Drive
    # shutil.copytree(source_folder, destination_folder)

    # # Print a success message
    # print(f"The 'runs' folder has been successfully updated in {destination_folder} in Google Drive.")

    # Get the best mAP value
    best_map = results.results_dict['metrics/mAP50-95(B)']

    # Save the best model
    if best_map > objective.best_value:
        objective.best_value = best_map
        print(f"BEST in {trial.number}")
        model.save(f'best_model_951_{trial.number}.pt')

    return best_map

def main():
    # Initialize best_value attribute
    objective.best_value = 0

    # Create a study object and optimize the objective function
    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner(),study_name='example-study', storage='sqlite:///example.db', load_if_exists=True)
    study.optimize(objective, n_trials=5)  # Reduced number of trials

    print('Best trial:')
    trial = study.best_trial
    print('  Value: ', trial.value)
    print('  Params: ')
    for key, value in trial.params.items():
        print('    {}: {}'.format(key, value))

    # Load and return the best model
    best_model_path = f'best_model_{trial.number}.pt'
    if os.path.exists(best_model_path):
        best_model = YOLO(best_model_path)
        print(f"Best model loaded from {best_model_path}")
    else:
        print("Best model file not found.")
if __name__ =='__main__':
    main()

[I 2024-07-14 18:20:05,541] Using an existing study with name 'example-study' instead of creating a new one.


New https://pypi.org/project/ultralytics/8.2.57 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.56 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov10m.pt, data=data.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_

train: Scanning /content/dataset/labels/train.cache... 2600 images, 2 backgrounds, 0 corrupt: 100%|██████████| 2600/2600 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/dataset/images/train/rainy day (236).jpg: 1 duplicate labels removed
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/dataset/labels/val.cache... 200 images, 0 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]


...

New https://pypi.org/project/ultralytics/8.2.57 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.56 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov10m.pt, data=data.yaml, epochs=8, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_t

train: Scanning /content/dataset/labels/train.cache... 2600 images, 2 backgrounds, 0 corrupt: 100%|██████████| 2600/2600 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/dataset/images/train/rainy day (236).jpg: 1 duplicate labels removed
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/dataset/labels/val.cache... 200 images, 0 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]


Plotting labels to runs/detect/train7/labels.jpg... 
optimizer: SGD(lr=5.3835214661932546e-05, momentum=0.937) with parameter groups 129 weight(decay=0.0), 142 weight(decay=0.0005), 141 bias(decay=0.0)


In [5]:
from google.colab import drive
import shutil
import os

# Mount Google Drive
#drive.mount('/content/drive')

# Define the source and destination paths
source_folder = '/content/runs'
destination_folder = '/content/drive/MyDrive/ML/runs'

# Check if the destination folder already exists
if os.path.exists(destination_folder):
    # Remove the existing folder in Google Drive to avoid conflicts
    shutil.rmtree(destination_folder)

# Copy the updated 'runs' folder to Google Drive
shutil.copytree(source_folder, destination_folder)

# Print a success message
print(f"The 'runs' folder has been successfully updated in {destination_folder} in Google Drive.")


The 'runs' folder has been successfully updated in /content/drive/MyDrive/ML/runs in Google Drive.


### TRAINING WITH ADAM OPTIMIZER

In [6]:
import optuna
from ultralytics import YOLO
from google.colab import drive
import shutil
import os

def objective(trial):
    # Define the hyperparameters to optimize
    hyperparameters = {
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-2),
        "epochs": trial.suggest_int("epochs", 5, 10),
        'batch_size': trial.suggest_categorical('batch_size', [8, 16]),
        'img_size': trial.suggest_categorical('img_size', [416, 512, 640]),
        'fliplr': trial.suggest_uniform('fliplr', 0.0, 0.5),
        'mosaic': trial.suggest_uniform('mosaic', 0.0, 1.0),
    }

    # Create the model
    model = YOLO('yolov10m.pt')  # or 'yolov8n.pt' for a pretrained model

    # Train the model
    results = model.train(
        data='data.yaml',
        epochs=hyperparameters['epochs'],
        imgsz=hyperparameters['img_size'],
        batch=hyperparameters['batch_size'],
        lr0=hyperparameters['learning_rate'],
        fliplr=hyperparameters['fliplr'],
        mosaic=hyperparameters['mosaic'],
        optimizer='Adam',
    )

    # source_folder = '/content/runs'
    # destination_folder = '/content/drive/My Drive/ML/runs'

    # # Check if the destination folder already exists
    # if os.path.exists(destination_folder):
    #     # Remove the existing folder in Google Drive to avoid conflicts
    #     shutil.rmtree(destination_folder)

    # # Copy the updated 'runs' folder to Google Drive
    # shutil.copytree(source_folder, destination_folder)

    # # Print a success message
    # print(f"The 'runs' folder has been successfully updated in {destination_folder} in Google Drive.")

    # Get the best mAP value
    best_map = results.results_dict['metrics/mAP50-95(B)']

    # Save the best model
    if best_map > objective.best_value:
        objective.best_value = best_map
        print(f"BEST in {trial.number}")
        model.save(f'best_model_952_{trial.number}.pt')

    return best_map

def main():
    # Initialize best_value attribute
    objective.best_value = 0

    # Create a study object and optimize the objective function
    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.MedianPruner(),study_name='example-study', storage='sqlite:///example.db', load_if_exists=True)
    study.optimize(objective, n_trials=5)  # Reduced number of trials

    print('Best trial:')
    trial = study.best_trial
    print('  Value: ', trial.value)
    print('  Params: ')
    for key, value in trial.params.items():
        print('    {}: {}'.format(key, value))

    # Load and return the best model
    best_model_path = f'best_model_{trial.number}.pt'
    if os.path.exists(best_model_path):
        best_model = YOLO(best_model_path)
        print(f"Best model loaded from {best_model_path}")
    else:
        print("Best model file not found.")
if __name__ =='__main__':
    main()

[I 2024-07-14 19:05:06,355] Using an existing study with name 'example-study' instead of creating a new one.


New https://pypi.org/project/ultralytics/8.2.57 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.56 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov10m.pt, data=data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=416, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train9, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, sav

train: Scanning /content/dataset/labels/train.cache... 2600 images, 2 backgrounds, 0 corrupt: 100%|██████████| 2600/2600 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/dataset/images/train/rainy day (236).jpg: 1 duplicate labels removed
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/dataset/labels/val.cache... 200 images, 0 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]


...


6 epochs completed in 0.179 hours.
Optimizer stripped from runs/detect/train13/weights/last.pt, 33.5MB
Optimizer stripped from runs/detect/train13/weights/best.pt, 33.5MB

Validating runs/detect/train13/weights/best.pt...
Ultralytics YOLOv8.2.56 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv10m summary (fused): 369 layers, 16,466,596 parameters, 0 gradients, 63.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:09<00:00,  1.37s/it]


                   all        200       1199      0.659       0.38      0.453      0.225
                   car        159        743       0.72      0.791      0.798      0.367
                  bike         99        125      0.595      0.416      0.459       0.24
                  auto         47         60      0.479      0.433      0.426      0.171
                 cycle         29         46      0.308     0.0652     0.0907     0.0502
                   bus         48         52      0.791      0.581      0.675      0.325
             minitruck         54         58      0.597      0.102      0.142      0.053
                 truck         32         47      0.767      0.489      0.723      0.368
                   van         25         28      0.334     0.0357     0.0996     0.0474
                  taxi         10         10          1      0.886      0.902      0.547
                  toto         28         30          1          0      0.219     0.0821
Speed: 0.7ms preproce

[I 2024-07-14 20:11:14,776] Trial 13 finished with value: 0.2250185117159155 and parameters: {'learning_rate': 0.00043873738629791816, 'epochs': 6, 'batch_size': 16, 'img_size': 512, 'fliplr': 0.16772357173645847, 'mosaic': 0.655474085609502}. Best is trial 12 with value: 0.27340935385702486.


Best trial:
  Value:  0.27340935385702486
  Params: 
    learning_rate: 0.0003422729959319013
    epochs: 8
    batch_size: 8
    img_size: 512
    fliplr: 0.013579357108739143
    mosaic: 0.0862606022494592
Best model file not found.


In [7]:
from google.colab import drive
import shutil
import os

# Mount Google Drive
#drive.mount('/content/drive')

# Define the source and destination paths
source_folder = '/content/runs'
destination_folder = '/content/drive/MyDrive/ML/runs'

# Check if the destination folder already exists
if os.path.exists(destination_folder):
    # Remove the existing folder in Google Drive to avoid conflicts
    shutil.rmtree(destination_folder)

# Copy the updated 'runs' folder to Google Drive
shutil.copytree(source_folder, destination_folder)

# Print a success message
print(f"The 'runs' folder has been successfully updated in {destination_folder} in Google Drive.")


The 'runs' folder has been successfully updated in /content/drive/MyDrive/ML/runs in Google Drive.
